In [28]:
from sqlalchemy import insert
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import MetaData

metadata_obj = MetaData()
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

from sqlalchemy import text
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///db_1.db")
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

In [2]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [3]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

(1, 'spongebob', 'Spongebob Squarepants')
(4, 'spongebob', 'Spongebob Squarepants')


In [6]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import ForeignKey
from sqlalchemy.orm import Session

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [7]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
(User(id=4, name='spongebob', fullname='Spongebob Squarepants'),)


In [13]:
row[0]

User(id=1, name='spongebob', fullname='Spongebob Squarepants')

In [16]:
row = session.execute(select(User.name, User.fullname)).first()
row

('spongebob', 'Spongebob Squarepants')

In [18]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(
        insert(address_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

In [29]:
from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {
                "username": "spongebob",
                "email_address": "spongebob@sqlalchemy.org",
            },
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

In [30]:
session.execute(
    select(User.name, Address).where(User.id == Address.user_id).order_by(Address.id)
).all()

[('spongebob', Address(id=1, email_address='spongebob@sqlalchemy.org')),
 ('sandy', Address(id=2, email_address='sandy@sqlalchemy.org')),
 ('sandy', Address(id=3, email_address='sandy@squirrelpower.org'))]

In [ ]:
from sqlalchemy import func, cast
stmt = select(
    ("Username: " + user_table.c.name).label("username"),
).order_by(user_table.c.name)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")
# BEGIN (implicit)
# SELECT ? || user_account.name AS username
# FROM user_account ORDER BY user_account.name
# [...] ('Username: ',)

Username: patrick
Username: patrick
Username: patrick
Username: patrick
Username: sandy
Username: sandy
Username: sandy
Username: sandy
Username: spongebob
Username: spongebob
Username: spongebob


In [ ]:
from sqlalchemy import text
stmt = select(text("'some phrase'"), user_table.c.name).order_by(user_table.c.name)
with engine.connect() as conn:
    print(conn.execute(stmt).all())
# BEGIN (implicit)
# SELECT 'some phrase', user_account.name
# FROM user_account ORDER BY user_account.name
# [generated in ...] ()

[('some phrase', 'patrick'), ('some phrase', 'patrick'), ('some phrase', 'patrick'), ('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'sandy'), ('some phrase', 'sandy'), ('some phrase', 'sandy'), ('some phrase', 'spongebob'), ('some phrase', 'spongebob'), ('some phrase', 'spongebob')]


In [ ]:
from sqlalchemy import literal_column
stmt = select(literal_column("'some phrase'").label("p"), user_table.c.name).order_by(
    user_table.c.name
)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")

# BEGIN (implicit)
# SELECT 'some phrase' AS p, user_account.name
# FROM user_account ORDER BY user_account.name
# [generated in ...] ()

some phrase, patrick
some phrase, patrick
some phrase, patrick
some phrase, patrick
some phrase, sandy
some phrase, sandy
some phrase, sandy
some phrase, sandy
some phrase, spongebob
some phrase, spongebob
some phrase, spongebob
